##Configuration Variables for Dana Birmingham ASC1 line

This script contains configuration details needed to transform data from the Dana Birmingham ASC1 line into our data cube format. Although the configuration details are hard-coded in this prototype, they will be stored in a database and loaded during runtime in production. This configuration based approach ensures flexibility towards a range of diversely formatted source data. 

###Configuration Variables defined in this script:

1. configuration and authentication info to connect to Kafka topic on Confluent cluster
2. the assembly line for which the cube is generated
3. the target test station for Root Cause Analysis (RCA)
4. the check point location, output directory, and trigger interval for the data stream
5. the output directory of DF_RUN (for RCA)
6. the technical schema for Kafka messages
7. the sensor measurement data schema
8. the business logic mapping
9. sensor mapping table created from a csv file
10. the pivot table schema and pivotted features list

In [0]:
import pyspark.sql.types as T
import json

### configuration details to connect to Kafka source on Confluent cluster
conf={'bootstrap.servers': 'pkc-4rn2p.canadacentral.azure.confluent.cloud:9092', 
      'security.protocol': 'SASL_SSL', 
      'sasl.mechanisms': 'PLAIN', 
      'sasl.username': 'D27IHIL45XD46XTF', 
      'sasl.password': 'UB3NYoxYI1NYvLMUEZrHuu5nYO9ZFR4jwAJMxQckb10QxvWtjU3zP1363Y2Akgcg',
      'startingOffsets':'earliest',
      #'startingOffsets':'{"birmingham_asc1_data":{"0":23}}',
      'topic':'birmingham_asc1_data'
     }


# NOTE: the following attributes should be loaded from a sql table or config file during production
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# line and target station for the transformation workflow
line = "Dana_ASC1"
target = 'op_280'

# transformation pipeline specs and output locations
checkpointPath = 'dbfs:/acerta/checkpoint/test-stream-1/'
triggerProcessingInterval = "30 seconds"
outputStagingPath = f'dbfs:/acerta/output/cube/staging/{line}/'
outputPivotPath = f'dbfs:/acerta/output/cube/pivot/{line}/'
outputPathDir = f'dbfs:/acerta/output/cube/{line}/'
inputPathDir = f'dbfs:/acerta/output/cube/{line}/'
outputDfRunDir = f'dbfs:/acerta/output/rca/run/{line}/'
outputDfRunHistoryDir = f'dbfs:/acerta/output/rca/history/{line}/'


### define data schema and business logic mapping configurations 
# ingress data configuration
ingress_schema_ddl = "`dataSourceId` STRING, `sourceFileId` STRING, `schemaVersion` STRING, `headers` ARRAY<STRING>, `data` ARRAY<ARRAY<STRING>>"
ingress_schema = T._parse_datatype_string(ingress_schema_ddl)

# sensor data configuration (for ASC1 line)
sensor_data_schema_ddl = "`serial_number` STRING, `process_name` STRING, `process_attribute_1` STRING, `process_attribute_2` STRING, `process_attribute_3` STRING, `process_attribute_4` STRING, `process_attribute_5` STRING, `data_element_name` STRING, `data_element_attribute_1` STRING, `data_element_attribute_2` STRING, `data_element_attribute_3` STRING, `data_element_attribute_4` STRING, `data_element_attribute_5` STRING, `location_name` STRING, `parent_location_name` STRING, `part_number` STRING, `data_value` STRING, `created` STRING, `trace` STRING, `test` STRING"
sensor_data_logic_mapping = '{"line":["parent_location_name"], \
                              "station_config":["location_name"], \
                              "sensor_config":["parent_location_name", "location_name", "process_name", "process_attribute_1", "process_attribute_2", "process_attribute_3", "process_attribute_4", "process_attribute_5", "data_element_name", "data_element_attribute_1", "data_element_attribute_2", "data_element_attribute_3", "data_element_attribute_4", "data_element_attribute_5"], \
                              "part_number": ["part_number"], \
                              "serial_number": ["serial_number"], \
                              "measurement":["data_value"], \
                              "measured_time":"created"}'
logic_mapping_json = json.loads(sensor_data_logic_mapping)


### Load sensor to uuid mapping table
sensorDf = spark.read.option("header",True).csv('/FileStore/tables/sensor_mapping.csv')


### define pivot schema ddl and pivot columns for ASC1 line
pivot_schema_ddl = "`part_number` STRING, `serial_number` STRING, `timestamp` STRING, `station` STRING, `003a5adb-84ac-42cc-8a40-8ab769e37793_data_value` STRING, `0135029f-e345-4a01-835a-c9758b114422_data_value` STRING, `01c7ae65-eb2f-4c3b-adda-47909f3607e0_data_value` STRING, `01ed8563-f7df-48ba-8f9c-931a770256c6_data_value` STRING, `0216235c-85b4-4667-85d6-53a9fa0f6497_data_value` STRING, `029ef063-639d-43f6-b54f-4d94ab869600_data_value` STRING, `03af25ac-7967-41ae-b6ff-917fe9205358_data_value` STRING, `04fa520c-4192-4c4b-9a32-1a4e2d49abc9_data_value` STRING, `05cdcc75-b1be-4e59-9893-47c7ec54eb6c_data_value` STRING, `05ee1df9-6446-4af9-a7a9-d4c86ce158f8_data_value` STRING, `06559036-4fd0-45d0-9d58-47c539e52df1_data_value` STRING, `07ab9bef-b94b-4416-9334-529830aec9c5_data_value` STRING, `0804e486-ea4e-4904-a813-abc8f92c406b_data_value` STRING, `080808ff-06f4-4735-baf2-284dc679b258_data_value` STRING, `09025ee0-8315-4be6-b39c-22cea2d5160b_data_value` STRING, `0ad7a34f-8dcb-4ffb-a004-73527b40ef42_data_value` STRING, `0b1cc335-ed4e-419a-84ac-ee58b6d7cc16_data_value` STRING, `0b34d0b6-e25a-43b3-9115-2aaff8fe5340_data_value` STRING, `0cdc5a0f-fc46-49af-9481-16a990b0edd5_data_value` STRING, `0cf2e4fb-f4a1-4209-a36a-47f47854f819_data_value` STRING, `0d1fa13b-3561-4aaf-96af-90533a1d78cc_data_value` STRING, `101df26c-c76f-4d3a-8c3f-80a1cb8d86ef_data_value` STRING, `1038dccf-2c9b-4f24-a90c-7796ec503915_data_value` STRING, `13371f7e-71b3-44bd-9f07-3830a1e04111_data_value` STRING, `13c80c14-6567-4563-8473-c5184348246d_data_value` STRING, `1634fe18-9cca-4c35-9bb5-7a14a919e063_data_value` STRING, `173005c3-33c5-453e-9e79-fe6df32f0538_data_value` STRING, `1805a47f-00ff-453e-bc40-ab0efcca70ad_data_value` STRING, `1988409c-2baa-4966-893e-0d32c8ff15c1_data_value` STRING, `1991a9ca-1266-4b26-8cb5-8f19eaa29b16_data_value` STRING, `19d443db-2b71-4b4a-a076-d0de8a39f731_data_value` STRING, `19f5704f-5462-4b37-a006-28f6242bd5f9_data_value` STRING, `1b3bdcfd-12e3-4f08-803a-2e4d205ed8fb_data_value` STRING, `1ba8ab6b-8c02-4188-8a8a-190b47f8a3eb_data_value` STRING, `1c210ffd-e9ed-4e54-8e12-c08900e56d32_data_value` STRING, `1cc4335e-e29c-45f2-9b4b-3b131275385e_data_value` STRING, `1d39fcb7-415f-4c0a-b641-5d46cdb7c912_data_value` STRING, `1db9643b-f8af-45dd-8a37-684594b977d2_data_value` STRING, `1fd465ba-9960-4327-8eb6-b107c36b15f9_data_value` STRING, `206ba4a7-52f1-4e8f-809d-9e63be30cac0_data_value` STRING, `222973a3-6201-4d6f-a743-8f60b2394b5a_data_value` STRING, `222c4e58-c585-4b06-a64c-3e3487afe358_data_value` STRING, `237f76f4-68c5-426b-99c5-57d00ac0f7c1_data_value` STRING, `24388bc3-5953-4477-9aec-34baba54910c_data_value` STRING, `257505e2-b113-4021-92a7-430a9695ba75_data_value` STRING, `284cd4cd-9d0a-4055-ba4c-bf29ebe01526_data_value` STRING, `2abc23a2-dfa1-4d7a-9355-10bf3bdfcead_data_value` STRING, `2b3202d0-e0f1-4a12-b8cc-c60ab5df716e_data_value` STRING, `2c329b38-d4da-414a-bc9c-dc61a543bd8b_data_value` STRING, `2c85942f-e4a4-415c-bf02-19cc67043356_data_value` STRING, `2cd12730-d856-45df-a71c-f76e44e65906_data_value` STRING, `2ddce6ed-e684-416c-ac27-62aa0e623939_data_value` STRING, `2eb47804-3c56-4c95-b2d6-9b4ff5ea805d_data_value` STRING, `2fc7b965-7f82-4a1c-8aea-ca67c991b961_data_value` STRING, `31848ef6-70ca-4581-b0ff-3498df7ebd7d_data_value` STRING, `32222459-650f-4855-ace1-322e395b5bb1_data_value` STRING, `329353ca-90d3-4f9b-819f-8da5e4f85b93_data_value` STRING, `33b95cc6-a368-45e9-8fec-2e29917b4cd1_data_value` STRING, `3538ccc1-cb81-4001-ad38-37f368137167_data_value` STRING, `35481747-9994-4e31-af81-bc3ca4c7320a_data_value` STRING, `35823798-dee4-4bef-8215-a49e22146c9c_data_value` STRING, `37bb5ee2-0522-4fed-8617-dae672b3a53c_data_value` STRING, `3848c376-62a7-4d5a-adb2-04ec8299e9d4_data_value` STRING, `38e08747-be0e-45f2-b2e3-aada414dc1b3_data_value` STRING, `39cc77c1-e11b-4867-ae60-a16089184bb9_data_value` STRING, `3a3c8335-11d4-493d-a172-875ace4fbb67_data_value` STRING, `3aa1943c-4e51-49e7-a203-cde4c5e85fd6_data_value` STRING, `3b87d8c1-6963-49c0-b100-bcab7a05a8a4_data_value` STRING, `3c532eb3-f10e-450a-b71c-ece6add00546_data_value` STRING, `3d99d1ef-8344-408e-8461-c36596e97c64_data_value` STRING, `3f033066-fde1-46e2-889e-e34b27edf212_data_value` STRING, `3fc6baf6-ce78-41cd-92a2-8b8890c25a3b_data_value` STRING, `3fc709f6-7ede-4bbd-9628-074e81d203c9_data_value` STRING, `419a8623-8796-47e5-a850-fb1b567354b8_data_value` STRING, `4543750c-fbea-414d-b512-becc15507c1f_data_value` STRING, `4551dd59-2399-4ac8-8c56-20670c96e5c9_data_value` STRING, `4637b1ee-6ac3-495a-8023-c6eaae4b9b7b_data_value` STRING, `463bc312-dbeb-4742-a842-9265bc9092d7_data_value` STRING, `4807e26e-c867-4112-b5ba-eb5c3ed88a2c_data_value` STRING, `4d59dc01-7e81-41d7-906a-d616e6547320_data_value` STRING, `4ea75379-5411-4a71-98de-2996e5dcb000_data_value` STRING, `572a7408-d5e7-409e-89c5-64b9efb21ed0_data_value` STRING, `5a2cdf99-b959-4343-bd3a-a071c6a35318_data_value` STRING, `5a3cfd9f-4aac-4687-8ee2-d6478921e677_data_value` STRING, `5bc0b349-3d37-4a04-898d-f2668feae94c_data_value` STRING, `5e2e0216-80b0-4855-9862-3b6b029e259f_data_value` STRING, `5e8d3b73-3636-4ee4-a807-5b8639b4b344_data_value` STRING, `5f2e6c91-93f4-469f-be2b-25bd6dc7f3ea_data_value` STRING, `5fe26633-a360-4335-8d91-f3a8283b14e2_data_value` STRING, `62a79d8c-644c-44c6-99cc-3b7c8312269b_data_value` STRING, `6334261c-5f3a-4fc9-9bef-41ab374a919c_data_value` STRING, `633d8d76-8f64-4b8d-8e34-d1a2dbe49767_data_value` STRING, `63e3913e-64ec-4b44-af1c-f2d14c0c7d7c_data_value` STRING, `644c581e-daa9-4e03-9160-4f9f4d38bf02_data_value` STRING, `6971b25b-c706-40d3-a595-cbb86a919437_data_value` STRING, `6b4b319b-72b4-4300-9825-4011c73a4397_data_value` STRING, `6be803f9-3230-4803-a77b-2bb4853eb63a_data_value` STRING, `6daffd74-4ac0-4673-aceb-246933e37615_data_value` STRING, `6dc1e18a-282d-4c1c-a8a4-56aaf8df95ad_data_value` STRING, `6e77b526-4e21-4a1a-af29-d8bdf698ae3e_data_value` STRING, `6fff0def-3328-4e3d-b4a4-6f2701c696b9_data_value` STRING, `701bd0a8-8d59-4e71-b6ab-1b5d0b91865f_data_value` STRING, `7085b7a1-9a65-445c-8dff-99a0502100f0_data_value` STRING, `7099d3c3-528a-48d6-937a-16f75f974268_data_value` STRING, `71a775d5-1730-4707-9f7d-b1b55881655d_data_value` STRING, `739f8399-f374-4b17-9a7f-a11c641dbfd0_data_value` STRING, `74959cd4-6cac-4aa6-b5df-b4ca71897933_data_value` STRING, `74a9f736-b297-4399-b0ae-6f954ec25e50_data_value` STRING, `75ae3b12-2b07-49a4-94d7-be033604194b_data_value` STRING, `76e3f5e5-c71e-416d-92cc-8dfc9e70e33a_data_value` STRING, `7702d172-5b07-4249-94f1-ce0431502425_data_value` STRING, `774a1582-5d2d-4e8f-a1b3-1921436657c8_data_value` STRING, `78317c3d-65b8-45e0-99fb-80179bdd67cf_data_value` STRING, `785ddb70-d53d-4e0d-ab2c-2a0e9c8e78e6_data_value` STRING, `7a054107-cb39-4f0b-a04a-56b4fcc1b5c8_data_value` STRING, `7a423822-942f-47f0-9438-4b76404b8b54_data_value` STRING, `7a626533-415e-4140-a452-37f4e6d5b37d_data_value` STRING, `7a983867-3360-4430-9041-83a9788d234e_data_value` STRING, `7b9460d5-fda5-473e-85b9-f26d6472bd7f_data_value` STRING, `7c0aa098-e794-4c3d-ae71-b0a3e9c27960_data_value` STRING, `7d4a1b4d-493e-40e4-87ae-1b34f2d8399f_data_value` STRING, `7da7ba03-4a01-4df5-b0b5-ae6c86477ade_data_value` STRING, `7dbcbc82-1311-4bd9-aa87-f60e4c613235_data_value` STRING, `7f6a6773-3f98-4f25-b42c-d80d69c1c182_data_value` STRING, `7f9bfc34-b033-4a44-8237-e78b373d47ce_data_value` STRING, `8040bd1b-d3da-438a-86e3-4422304ac5cb_data_value` STRING, `81a4ba95-a9cb-43b7-8879-c7772d7bcab9_data_value` STRING, `820f1642-4aa0-4942-9f15-7469ad4ece53_data_value` STRING, `8256d12b-efbd-406e-926c-2a765a70ab9b_data_value` STRING, `84253d13-d2e8-4a12-a3b8-138cab495271_data_value` STRING, `848b917b-bf89-4d8f-be58-38aa7b584af6_data_value` STRING, `85ccf098-9ba1-4290-909c-3921106ded21_data_value` STRING, `85f31b2f-9f41-450e-8e4d-ae8c2d14cafa_data_value` STRING, `86013866-c5f5-4622-9359-f8c37735e2c6_data_value` STRING, `8646babc-41df-4558-9b7d-528b3c25b2fe_data_value` STRING, `886e7f2f-ba76-4328-9fab-b140c6b5bfe6_data_value` STRING, `88cf4a1c-3bef-4f4e-825a-e3b6b735d8a1_data_value` STRING, `8ab56b7f-cd14-44dd-bd59-f40265c5e9fd_data_value` STRING, `8c562a2e-f70c-4431-a8d3-aa093ba98c8d_data_value` STRING, `8d71be6e-6534-4ad0-8660-5da5d6f8c71a_data_value` STRING, `8f77b68d-85d9-4304-be91-a2c21041a582_data_value` STRING, `8f822ca7-9df3-4ee2-9b4e-61c42935a8db_data_value` STRING, `90a43220-88c8-47a5-a5f6-7400d05bb16d_data_value` STRING, `91258bda-0c61-427f-9dbd-b2e3ae6fae4b_data_value` STRING, `93272972-fcbf-44e0-ba6b-9ab2553f8f55_data_value` STRING, `93d79426-dd2b-45ad-bdf6-50f968bc7ade_data_value` STRING, `99ff71f0-a0cb-4bfc-b280-3d8b4f08d692_data_value` STRING, `9a3a4e09-cc1e-4ecb-b7ea-09ad7af3fb65_data_value` STRING, `9b443739-8c8e-45bf-bafd-e66fc0bf8867_data_value` STRING, `9f3328d0-2afc-4a12-884e-5f54d559c8ca_data_value` STRING, `a011f4cf-5650-4c1e-8aaa-b09cb17db7dc_data_value` STRING, `a0e403ab-baf1-483c-adb2-0f87e18401f6_data_value` STRING, `a194dd80-35ee-4b17-86b2-047d4eb6e39a_data_value` STRING, `a1e63613-a854-45f3-b041-f10a266854ee_data_value` STRING, `a44efa1b-a7f4-4491-af22-7dfda53927d7_data_value` STRING, `a6b404e1-b6b5-4db9-8f6a-8850271779c8_data_value` STRING, `a6fd32b0-f7d7-4cc2-84fd-a62946ad7dc6_data_value` STRING, `a70ff64b-7118-4985-bb91-315f69a7358d_data_value` STRING, `a7ade523-ac28-4743-9560-0d05398ef836_data_value` STRING, `a8706704-38de-4166-8d71-bbebf2bb1b7a_data_value` STRING, `a937a64b-246c-49be-ad20-431f4824bc0b_data_value` STRING, `aadc83c9-8ee3-4ed6-94b1-601b7e1e79d3_data_value` STRING, `aafad8bf-6612-4a72-93be-c6ff22ecff86_data_value` STRING, `ab0565dd-9a38-40d3-8bec-e9b63ab8275f_data_value` STRING, `ab691333-74d1-4e94-bf15-de38c0734018_data_value` STRING, `ad20f626-9bdc-44e3-a6ca-c3b8dc71b3e8_data_value` STRING, `ae490f0f-cbd1-4536-96eb-36e3bd02f6bd_data_value` STRING, `ae9158de-1516-4190-ad14-13780339ea1a_data_value` STRING, `aedcbab9-6a75-4cbb-8189-55836e8e3bf6_data_value` STRING, `b0b3a3de-6a11-4e58-bf35-c5b4336f4cda_data_value` STRING, `b0cfa64a-f474-4b0f-87b8-843405d55678_data_value` STRING, `b3c82613-e251-4eae-89bf-0f028e5f9b67_data_value` STRING, `b3d7464c-9502-4c5d-9a86-9419f4c239d8_data_value` STRING, `b3f82972-32c6-4b45-93e9-4a596e3cb342_data_value` STRING, `b6731eba-0f6c-44de-a80f-c0aea0b5c6ab_data_value` STRING, `b8102fbf-d811-43b3-ba43-7e941d321ef0_data_value` STRING, `b8ccfaa5-4915-4094-a47f-26ef6bf139be_data_value` STRING, `bae0b1d5-3d52-4bd3-91f1-b2d669707b4d_data_value` STRING, `bb8b4249-e715-4d9c-b61a-d5c04132b8e4_data_value` STRING, `bbc9061c-9e91-4118-8223-5532e853df28_data_value` STRING, `bbdd947d-c747-48ac-a8a1-f523bb764202_data_value` STRING, `bbf1251b-c6b5-4c30-a193-7dc2a5b006b5_data_value` STRING, `bcb8b603-3c22-4648-85f1-1d19f406e790_data_value` STRING, `bcb9883c-352b-49f0-9692-fee664c37d98_data_value` STRING, `bcf4f21b-5c26-4145-8bb1-55a5201d48fa_data_value` STRING, `bdf73670-c821-4971-8fb8-832871ea0f07_data_value` STRING, `bf3c54db-09b7-46cc-b08b-244b928c6daf_data_value` STRING, `bfa64061-a7d8-479a-8e1f-9ef1059a3c5c_data_value` STRING, `c12080a3-dc96-4fff-968a-0861aae6999a_data_value` STRING, `c1c633ea-bf38-4cc3-93e0-5a6f0be4e33d_data_value` STRING, `c2de308e-bc92-4aea-84f4-38c1b657f8bc_data_value` STRING, `c3d2d832-cab3-46f7-a59e-c37f46150896_data_value` STRING, `c50eb4ba-8ad9-4bbf-befd-9030f7112d56_data_value` STRING, `c54bd411-d8c4-4f5e-8524-64b191c1f2ac_data_value` STRING, `c575010e-f7d7-467f-ad28-4352612f1866_data_value` STRING, `c5a89c9d-1f1d-414d-9ebf-f9c6da5b8946_data_value` STRING, `c698ecc2-a69b-4f72-b263-322a5c558308_data_value` STRING, `c6d9f0ae-bb58-4621-b852-9ed5a408075f_data_value` STRING, `c8197014-2e35-4c5a-a265-f732a7f1e7b0_data_value` STRING, `ca4f74a6-4164-440a-b2be-5c1277125027_data_value` STRING, `cac7e975-bce8-4e5c-9711-b2af838c0282_data_value` STRING, `cdd5370f-380b-426b-9035-5ae2ead91b99_data_value` STRING, `cf4eb270-95b6-4b6a-a75c-8de8f355379b_data_value` STRING, `d04f41d1-b722-4516-8637-8a801f0c87b4_data_value` STRING, `d3e1e0fa-1b76-4082-b5ea-5b13d2b2adcd_data_value` STRING, `d4156b68-758f-4681-83b6-fb06c1422e5b_data_value` STRING, `d464030d-b97f-4d3b-9d6a-19a3ca61495c_data_value` STRING, `d4e33ce1-c29c-4868-8319-f9d8a465ce96_data_value` STRING, `d6b7dba8-9a7b-4aa2-94de-d0faa93b3d5a_data_value` STRING, `da4220e8-79a7-4435-97b9-abb3427066a4_data_value` STRING, `da7482e3-b3ac-499b-b16f-9b9eb9a86eb6_data_value` STRING, `da87bce1-e885-4d43-b665-b301e005ddf0_data_value` STRING, `db09f536-1de1-46a9-8883-c862bd4f414e_data_value` STRING, `ddcaf739-fdac-4703-b722-9ee4d33c59df_data_value` STRING, `de825f32-2561-48f3-85f6-5aa057ccd5ae_data_value` STRING, `df190c4b-8ac0-4b19-9d1c-4d3f73e9ac45_data_value` STRING, `e10d7397-6bc4-4a63-8b28-f273a4995f82_data_value` STRING, `e4e2fe21-b4c9-4314-ac2f-d69ca6a42ef8_data_value` STRING, `e8df2af3-2a25-4ab3-8a90-cbd99c333b35_data_value` STRING, `ea0cecac-63ed-4894-b31d-0b52c62a9d19_data_value` STRING, `ea3d4d4e-f659-441a-b92b-4f3f4511216b_data_value` STRING, `ea47525b-983e-43db-825b-9364ff2f402e_data_value` STRING, `ea97339d-f6f3-4381-8445-8fc7a95649f5_data_value` STRING, `eb83c74d-6b6d-4564-9f03-ac352a24572d_data_value` STRING, `eba0cdd2-a265-4331-8e50-48295c1df80b_data_value` STRING, `ed231805-1a3b-46f9-8329-94bc5407218a_data_value` STRING, `eda2466b-0b53-4447-b663-5c7259c193ab_data_value` STRING, `edb6d1cf-938c-4d4c-95fb-e6d06930a473_data_value` STRING, `ee92e25d-81fa-421a-88f7-f620a26e8659_data_value` STRING, `f04032fe-38e7-449a-810f-e53fab5b66d9_data_value` STRING, `f06dad9e-5a88-47b0-859d-1be657f74b70_data_value` STRING, `f09d3861-c950-4c28-861f-16137e5e5131_data_value` STRING, `f252fba1-361c-4b0b-b329-bfd77f912ce6_data_value` STRING, `f2c3fe93-286f-4631-bfef-dc969b964059_data_value` STRING, `f2e44741-b2f5-4dfc-92f8-6c35829db11f_data_value` STRING, `f39db795-6ca8-437d-8c31-6e65468ba7d5_data_value` STRING, `f50cc510-0f2f-407a-971f-2395d03acc51_data_value` STRING, `f57af9e8-3f9a-4cdd-9d00-08f2f5b60397_data_value` STRING, `f6197054-e093-437f-b86b-de37a627b54d_data_value` STRING, `f67f87c6-0630-4b5f-9613-cc3ed2f2bce3_data_value` STRING, `f79d17a6-6c36-410f-8ec6-ce58fd84904b_data_value` STRING, `f82018ce-a08c-4192-b2c3-4223340d9653_data_value` STRING, `f8283dd9-c49d-4bfc-beeb-a6be0a1a26ab_data_value` STRING, `f839a0f7-cf4b-47fb-abdd-e45c8d9c46ef_data_value` STRING, `f844a50a-60a3-401d-9c0c-89be55c1b24e_data_value` STRING, `f853ae93-07c7-40f8-9563-fe5b02ef5c62_data_value` STRING, `f8f010b6-0c27-42b9-9c8a-baa34136a026_data_value` STRING, `f9e10d03-c0dd-4499-b5eb-bea92ea611c9_data_value` STRING, `fa0e0538-9ede-4455-ae45-a49b1b720fc6_data_value` STRING, `fa14669d-d1ca-43c6-a7cf-a16e158c1a6a_data_value` STRING, `fb2134b3-ce7e-4fee-ab4b-0c68ba3e85a1_data_value` STRING, `fbb80ce2-e9ed-41bb-9beb-f5449490e512_data_value` STRING, `fd5daf33-82ec-4764-8c82-f767afb3749f_data_value` STRING"

pivot_feature_cols = ["003a5adb-84ac-42cc-8a40-8ab769e37793_data_value", "0135029f-e345-4a01-835a-c9758b114422_data_value", "01c7ae65-eb2f-4c3b-adda-47909f3607e0_data_value", "01ed8563-f7df-48ba-8f9c-931a770256c6_data_value", "0216235c-85b4-4667-85d6-53a9fa0f6497_data_value", "029ef063-639d-43f6-b54f-4d94ab869600_data_value", "03af25ac-7967-41ae-b6ff-917fe9205358_data_value", "04fa520c-4192-4c4b-9a32-1a4e2d49abc9_data_value", "05cdcc75-b1be-4e59-9893-47c7ec54eb6c_data_value", "05ee1df9-6446-4af9-a7a9-d4c86ce158f8_data_value", "06559036-4fd0-45d0-9d58-47c539e52df1_data_value", "07ab9bef-b94b-4416-9334-529830aec9c5_data_value", "0804e486-ea4e-4904-a813-abc8f92c406b_data_value", "080808ff-06f4-4735-baf2-284dc679b258_data_value", "09025ee0-8315-4be6-b39c-22cea2d5160b_data_value", "0ad7a34f-8dcb-4ffb-a004-73527b40ef42_data_value", "0b1cc335-ed4e-419a-84ac-ee58b6d7cc16_data_value", "0b34d0b6-e25a-43b3-9115-2aaff8fe5340_data_value", "0cdc5a0f-fc46-49af-9481-16a990b0edd5_data_value", "0cf2e4fb-f4a1-4209-a36a-47f47854f819_data_value", "0d1fa13b-3561-4aaf-96af-90533a1d78cc_data_value", "101df26c-c76f-4d3a-8c3f-80a1cb8d86ef_data_value", "1038dccf-2c9b-4f24-a90c-7796ec503915_data_value", "13371f7e-71b3-44bd-9f07-3830a1e04111_data_value", "13c80c14-6567-4563-8473-c5184348246d_data_value", "1634fe18-9cca-4c35-9bb5-7a14a919e063_data_value", "173005c3-33c5-453e-9e79-fe6df32f0538_data_value", "1805a47f-00ff-453e-bc40-ab0efcca70ad_data_value", "1988409c-2baa-4966-893e-0d32c8ff15c1_data_value", "1991a9ca-1266-4b26-8cb5-8f19eaa29b16_data_value", "19d443db-2b71-4b4a-a076-d0de8a39f731_data_value", "19f5704f-5462-4b37-a006-28f6242bd5f9_data_value", "1b3bdcfd-12e3-4f08-803a-2e4d205ed8fb_data_value", "1ba8ab6b-8c02-4188-8a8a-190b47f8a3eb_data_value", "1c210ffd-e9ed-4e54-8e12-c08900e56d32_data_value", "1cc4335e-e29c-45f2-9b4b-3b131275385e_data_value", "1d39fcb7-415f-4c0a-b641-5d46cdb7c912_data_value", "1db9643b-f8af-45dd-8a37-684594b977d2_data_value", "1fd465ba-9960-4327-8eb6-b107c36b15f9_data_value", "206ba4a7-52f1-4e8f-809d-9e63be30cac0_data_value", "222973a3-6201-4d6f-a743-8f60b2394b5a_data_value", "222c4e58-c585-4b06-a64c-3e3487afe358_data_value", "237f76f4-68c5-426b-99c5-57d00ac0f7c1_data_value", "24388bc3-5953-4477-9aec-34baba54910c_data_value", "257505e2-b113-4021-92a7-430a9695ba75_data_value", "284cd4cd-9d0a-4055-ba4c-bf29ebe01526_data_value", "2abc23a2-dfa1-4d7a-9355-10bf3bdfcead_data_value", "2b3202d0-e0f1-4a12-b8cc-c60ab5df716e_data_value", "2c329b38-d4da-414a-bc9c-dc61a543bd8b_data_value", "2c85942f-e4a4-415c-bf02-19cc67043356_data_value", "2cd12730-d856-45df-a71c-f76e44e65906_data_value", "2ddce6ed-e684-416c-ac27-62aa0e623939_data_value", "2eb47804-3c56-4c95-b2d6-9b4ff5ea805d_data_value", "2fc7b965-7f82-4a1c-8aea-ca67c991b961_data_value", "31848ef6-70ca-4581-b0ff-3498df7ebd7d_data_value", "32222459-650f-4855-ace1-322e395b5bb1_data_value", "329353ca-90d3-4f9b-819f-8da5e4f85b93_data_value", "33b95cc6-a368-45e9-8fec-2e29917b4cd1_data_value", "3538ccc1-cb81-4001-ad38-37f368137167_data_value", "35481747-9994-4e31-af81-bc3ca4c7320a_data_value", "35823798-dee4-4bef-8215-a49e22146c9c_data_value", "37bb5ee2-0522-4fed-8617-dae672b3a53c_data_value", "3848c376-62a7-4d5a-adb2-04ec8299e9d4_data_value", "38e08747-be0e-45f2-b2e3-aada414dc1b3_data_value", "39cc77c1-e11b-4867-ae60-a16089184bb9_data_value", "3a3c8335-11d4-493d-a172-875ace4fbb67_data_value", "3aa1943c-4e51-49e7-a203-cde4c5e85fd6_data_value", "3b87d8c1-6963-49c0-b100-bcab7a05a8a4_data_value", "3c532eb3-f10e-450a-b71c-ece6add00546_data_value", "3d99d1ef-8344-408e-8461-c36596e97c64_data_value", "3f033066-fde1-46e2-889e-e34b27edf212_data_value", "3fc6baf6-ce78-41cd-92a2-8b8890c25a3b_data_value", "3fc709f6-7ede-4bbd-9628-074e81d203c9_data_value", "419a8623-8796-47e5-a850-fb1b567354b8_data_value", "4543750c-fbea-414d-b512-becc15507c1f_data_value", "4551dd59-2399-4ac8-8c56-20670c96e5c9_data_value", "4637b1ee-6ac3-495a-8023-c6eaae4b9b7b_data_value", "463bc312-dbeb-4742-a842-9265bc9092d7_data_value", "4807e26e-c867-4112-b5ba-eb5c3ed88a2c_data_value", "4d59dc01-7e81-41d7-906a-d616e6547320_data_value", "4ea75379-5411-4a71-98de-2996e5dcb000_data_value", "572a7408-d5e7-409e-89c5-64b9efb21ed0_data_value", "5a2cdf99-b959-4343-bd3a-a071c6a35318_data_value", "5a3cfd9f-4aac-4687-8ee2-d6478921e677_data_value", "5bc0b349-3d37-4a04-898d-f2668feae94c_data_value", "5e2e0216-80b0-4855-9862-3b6b029e259f_data_value", "5e8d3b73-3636-4ee4-a807-5b8639b4b344_data_value", "5f2e6c91-93f4-469f-be2b-25bd6dc7f3ea_data_value", "5fe26633-a360-4335-8d91-f3a8283b14e2_data_value", "62a79d8c-644c-44c6-99cc-3b7c8312269b_data_value", "6334261c-5f3a-4fc9-9bef-41ab374a919c_data_value", "633d8d76-8f64-4b8d-8e34-d1a2dbe49767_data_value", "63e3913e-64ec-4b44-af1c-f2d14c0c7d7c_data_value", "644c581e-daa9-4e03-9160-4f9f4d38bf02_data_value", "6971b25b-c706-40d3-a595-cbb86a919437_data_value", "6b4b319b-72b4-4300-9825-4011c73a4397_data_value", "6be803f9-3230-4803-a77b-2bb4853eb63a_data_value", "6daffd74-4ac0-4673-aceb-246933e37615_data_value", "6dc1e18a-282d-4c1c-a8a4-56aaf8df95ad_data_value", "6e77b526-4e21-4a1a-af29-d8bdf698ae3e_data_value", "6fff0def-3328-4e3d-b4a4-6f2701c696b9_data_value", "701bd0a8-8d59-4e71-b6ab-1b5d0b91865f_data_value", "7085b7a1-9a65-445c-8dff-99a0502100f0_data_value", "7099d3c3-528a-48d6-937a-16f75f974268_data_value", "71a775d5-1730-4707-9f7d-b1b55881655d_data_value", "739f8399-f374-4b17-9a7f-a11c641dbfd0_data_value", "74959cd4-6cac-4aa6-b5df-b4ca71897933_data_value", "74a9f736-b297-4399-b0ae-6f954ec25e50_data_value", "75ae3b12-2b07-49a4-94d7-be033604194b_data_value", "76e3f5e5-c71e-416d-92cc-8dfc9e70e33a_data_value", "7702d172-5b07-4249-94f1-ce0431502425_data_value", "774a1582-5d2d-4e8f-a1b3-1921436657c8_data_value", "78317c3d-65b8-45e0-99fb-80179bdd67cf_data_value", "785ddb70-d53d-4e0d-ab2c-2a0e9c8e78e6_data_value", "7a054107-cb39-4f0b-a04a-56b4fcc1b5c8_data_value", "7a423822-942f-47f0-9438-4b76404b8b54_data_value", "7a626533-415e-4140-a452-37f4e6d5b37d_data_value", "7a983867-3360-4430-9041-83a9788d234e_data_value", "7b9460d5-fda5-473e-85b9-f26d6472bd7f_data_value", "7c0aa098-e794-4c3d-ae71-b0a3e9c27960_data_value", "7d4a1b4d-493e-40e4-87ae-1b34f2d8399f_data_value", "7da7ba03-4a01-4df5-b0b5-ae6c86477ade_data_value", "7dbcbc82-1311-4bd9-aa87-f60e4c613235_data_value", "7f6a6773-3f98-4f25-b42c-d80d69c1c182_data_value", "7f9bfc34-b033-4a44-8237-e78b373d47ce_data_value", "8040bd1b-d3da-438a-86e3-4422304ac5cb_data_value", "81a4ba95-a9cb-43b7-8879-c7772d7bcab9_data_value", "820f1642-4aa0-4942-9f15-7469ad4ece53_data_value", "8256d12b-efbd-406e-926c-2a765a70ab9b_data_value", "84253d13-d2e8-4a12-a3b8-138cab495271_data_value", "848b917b-bf89-4d8f-be58-38aa7b584af6_data_value", "85ccf098-9ba1-4290-909c-3921106ded21_data_value", "85f31b2f-9f41-450e-8e4d-ae8c2d14cafa_data_value", "86013866-c5f5-4622-9359-f8c37735e2c6_data_value", "8646babc-41df-4558-9b7d-528b3c25b2fe_data_value", "886e7f2f-ba76-4328-9fab-b140c6b5bfe6_data_value", "88cf4a1c-3bef-4f4e-825a-e3b6b735d8a1_data_value", "8ab56b7f-cd14-44dd-bd59-f40265c5e9fd_data_value", "8c562a2e-f70c-4431-a8d3-aa093ba98c8d_data_value", "8d71be6e-6534-4ad0-8660-5da5d6f8c71a_data_value", "8f77b68d-85d9-4304-be91-a2c21041a582_data_value", "8f822ca7-9df3-4ee2-9b4e-61c42935a8db_data_value", "90a43220-88c8-47a5-a5f6-7400d05bb16d_data_value", "91258bda-0c61-427f-9dbd-b2e3ae6fae4b_data_value", "93272972-fcbf-44e0-ba6b-9ab2553f8f55_data_value", "93d79426-dd2b-45ad-bdf6-50f968bc7ade_data_value", "99ff71f0-a0cb-4bfc-b280-3d8b4f08d692_data_value", "9a3a4e09-cc1e-4ecb-b7ea-09ad7af3fb65_data_value", "9b443739-8c8e-45bf-bafd-e66fc0bf8867_data_value", "9f3328d0-2afc-4a12-884e-5f54d559c8ca_data_value", "a011f4cf-5650-4c1e-8aaa-b09cb17db7dc_data_value", "a0e403ab-baf1-483c-adb2-0f87e18401f6_data_value", "a194dd80-35ee-4b17-86b2-047d4eb6e39a_data_value", "a1e63613-a854-45f3-b041-f10a266854ee_data_value", "a44efa1b-a7f4-4491-af22-7dfda53927d7_data_value", "a6b404e1-b6b5-4db9-8f6a-8850271779c8_data_value", "a6fd32b0-f7d7-4cc2-84fd-a62946ad7dc6_data_value", "a70ff64b-7118-4985-bb91-315f69a7358d_data_value", "a7ade523-ac28-4743-9560-0d05398ef836_data_value", "a8706704-38de-4166-8d71-bbebf2bb1b7a_data_value", "a937a64b-246c-49be-ad20-431f4824bc0b_data_value", "aadc83c9-8ee3-4ed6-94b1-601b7e1e79d3_data_value", "aafad8bf-6612-4a72-93be-c6ff22ecff86_data_value", "ab0565dd-9a38-40d3-8bec-e9b63ab8275f_data_value", "ab691333-74d1-4e94-bf15-de38c0734018_data_value", "ad20f626-9bdc-44e3-a6ca-c3b8dc71b3e8_data_value", "ae490f0f-cbd1-4536-96eb-36e3bd02f6bd_data_value", "ae9158de-1516-4190-ad14-13780339ea1a_data_value", "aedcbab9-6a75-4cbb-8189-55836e8e3bf6_data_value", "b0b3a3de-6a11-4e58-bf35-c5b4336f4cda_data_value", "b0cfa64a-f474-4b0f-87b8-843405d55678_data_value", "b3c82613-e251-4eae-89bf-0f028e5f9b67_data_value", "b3d7464c-9502-4c5d-9a86-9419f4c239d8_data_value", "b3f82972-32c6-4b45-93e9-4a596e3cb342_data_value", "b6731eba-0f6c-44de-a80f-c0aea0b5c6ab_data_value", "b8102fbf-d811-43b3-ba43-7e941d321ef0_data_value", "b8ccfaa5-4915-4094-a47f-26ef6bf139be_data_value", "bae0b1d5-3d52-4bd3-91f1-b2d669707b4d_data_value", "bb8b4249-e715-4d9c-b61a-d5c04132b8e4_data_value", "bbc9061c-9e91-4118-8223-5532e853df28_data_value", "bbdd947d-c747-48ac-a8a1-f523bb764202_data_value", "bbf1251b-c6b5-4c30-a193-7dc2a5b006b5_data_value", "bcb8b603-3c22-4648-85f1-1d19f406e790_data_value", "bcb9883c-352b-49f0-9692-fee664c37d98_data_value", "bcf4f21b-5c26-4145-8bb1-55a5201d48fa_data_value", "bdf73670-c821-4971-8fb8-832871ea0f07_data_value", "bf3c54db-09b7-46cc-b08b-244b928c6daf_data_value", "bfa64061-a7d8-479a-8e1f-9ef1059a3c5c_data_value", "c12080a3-dc96-4fff-968a-0861aae6999a_data_value", "c1c633ea-bf38-4cc3-93e0-5a6f0be4e33d_data_value", "c2de308e-bc92-4aea-84f4-38c1b657f8bc_data_value", "c3d2d832-cab3-46f7-a59e-c37f46150896_data_value", "c50eb4ba-8ad9-4bbf-befd-9030f7112d56_data_value", "c54bd411-d8c4-4f5e-8524-64b191c1f2ac_data_value", "c575010e-f7d7-467f-ad28-4352612f1866_data_value", "c5a89c9d-1f1d-414d-9ebf-f9c6da5b8946_data_value", "c698ecc2-a69b-4f72-b263-322a5c558308_data_value", "c6d9f0ae-bb58-4621-b852-9ed5a408075f_data_value", "c8197014-2e35-4c5a-a265-f732a7f1e7b0_data_value", "ca4f74a6-4164-440a-b2be-5c1277125027_data_value", "cac7e975-bce8-4e5c-9711-b2af838c0282_data_value", "cdd5370f-380b-426b-9035-5ae2ead91b99_data_value", "cf4eb270-95b6-4b6a-a75c-8de8f355379b_data_value", "d04f41d1-b722-4516-8637-8a801f0c87b4_data_value", "d3e1e0fa-1b76-4082-b5ea-5b13d2b2adcd_data_value", "d4156b68-758f-4681-83b6-fb06c1422e5b_data_value", "d464030d-b97f-4d3b-9d6a-19a3ca61495c_data_value", "d4e33ce1-c29c-4868-8319-f9d8a465ce96_data_value", "d6b7dba8-9a7b-4aa2-94de-d0faa93b3d5a_data_value", "da4220e8-79a7-4435-97b9-abb3427066a4_data_value", "da7482e3-b3ac-499b-b16f-9b9eb9a86eb6_data_value", "da87bce1-e885-4d43-b665-b301e005ddf0_data_value", "db09f536-1de1-46a9-8883-c862bd4f414e_data_value", "ddcaf739-fdac-4703-b722-9ee4d33c59df_data_value", "de825f32-2561-48f3-85f6-5aa057ccd5ae_data_value", "df190c4b-8ac0-4b19-9d1c-4d3f73e9ac45_data_value", "e10d7397-6bc4-4a63-8b28-f273a4995f82_data_value", "e4e2fe21-b4c9-4314-ac2f-d69ca6a42ef8_data_value", "e8df2af3-2a25-4ab3-8a90-cbd99c333b35_data_value", "ea0cecac-63ed-4894-b31d-0b52c62a9d19_data_value", "ea3d4d4e-f659-441a-b92b-4f3f4511216b_data_value", "ea47525b-983e-43db-825b-9364ff2f402e_data_value", "ea97339d-f6f3-4381-8445-8fc7a95649f5_data_value", "eb83c74d-6b6d-4564-9f03-ac352a24572d_data_value", "eba0cdd2-a265-4331-8e50-48295c1df80b_data_value", "ed231805-1a3b-46f9-8329-94bc5407218a_data_value", "eda2466b-0b53-4447-b663-5c7259c193ab_data_value", "edb6d1cf-938c-4d4c-95fb-e6d06930a473_data_value", "ee92e25d-81fa-421a-88f7-f620a26e8659_data_value", "f04032fe-38e7-449a-810f-e53fab5b66d9_data_value", "f06dad9e-5a88-47b0-859d-1be657f74b70_data_value", "f09d3861-c950-4c28-861f-16137e5e5131_data_value", "f252fba1-361c-4b0b-b329-bfd77f912ce6_data_value", "f2c3fe93-286f-4631-bfef-dc969b964059_data_value", "f2e44741-b2f5-4dfc-92f8-6c35829db11f_data_value", "f39db795-6ca8-437d-8c31-6e65468ba7d5_data_value", "f50cc510-0f2f-407a-971f-2395d03acc51_data_value", "f57af9e8-3f9a-4cdd-9d00-08f2f5b60397_data_value", "f6197054-e093-437f-b86b-de37a627b54d_data_value", "f67f87c6-0630-4b5f-9613-cc3ed2f2bce3_data_value", "f79d17a6-6c36-410f-8ec6-ce58fd84904b_data_value", "f82018ce-a08c-4192-b2c3-4223340d9653_data_value", "f8283dd9-c49d-4bfc-beeb-a6be0a1a26ab_data_value", "f839a0f7-cf4b-47fb-abdd-e45c8d9c46ef_data_value", "f844a50a-60a3-401d-9c0c-89be55c1b24e_data_value", "f853ae93-07c7-40f8-9563-fe5b02ef5c62_data_value", "f8f010b6-0c27-42b9-9c8a-baa34136a026_data_value", "f9e10d03-c0dd-4499-b5eb-bea92ea611c9_data_value", "fa0e0538-9ede-4455-ae45-a49b1b720fc6_data_value", "fa14669d-d1ca-43c6-a7cf-a16e158c1a6a_data_value", "fb2134b3-ce7e-4fee-ab4b-0c68ba3e85a1_data_value", "fbb80ce2-e9ed-41bb-9beb-f5449490e512_data_value", "fd5daf33-82ec-4764-8c82-f767afb3749f_data_value"]
